In [ ]:
%cd ../..

### connect to your Gdrive

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

# go to the parent folder of mmk/

%cd /gdrive/My\ Drive/

### Install `mmk` and its dependencies

In [ ]:
!git clone https://github.com/antoinedaurat/mmk.git
!pip install -r mmk/requirements.txt

### Imports

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd
plt.rcParams['figure.figsize'] = (20, 6)
import torch
import numpy as np
import os

# class defining the class of FreqNet models and the dictionary to subclass it
from mmk.models.freqnet import FreqNet, layer_funcs

# generate function 
from mmk.modules.generate import generate

# data utils
from mmk.data import make_root_db, Database

# debug, interact
from mmk.utils import show, audio


### Data

In [ ]:
# where you have audio files 
my_music_folder = "nessun_dorma/data/arie/"

# the file with your data :
my_db = ".arie_mmk.h5"

if not os.path.exists(my_db):
    make_root_db(my_db, my_music_folder)

db = Database(my_db)

db.metadata

# Model's Definition and Training

## FreqNet's Parameters :

### Layer Functions

`FreqNet` is, similar to Wavenet, a model with stacked dilated convolution layer. The layer function of FreqNet allows you to specify different options for the implemententation of those stacked dilated convolution layers that result in models with different properties.

Options are :
- `strict` can be `True` or `False`. It specifies whether each layer is to be defined as an autoregressive function or not. Originally, Wavenet considers the whole network to be such a function. Accordingly, for an input of the length of wavenet's receptive-field, wavenet outputs a single (future) time-step. Setting `strict` to `True`, makes each layer outputs a (future) time-step, which results in an output of `#of-layers` time-steps for an input with length equal to the receptive-field. 
- `accum_outputs` adds residual to the layer's definition. Possible values are `[-1, 0, 1]` for residuals aligned to the left, no residuals, and residuals aligned to the right respectively.
- `concat_outputs` also takes `[-1, 0, 1]` as possible values for left-, None and right-concatenation respectively. Since convolution layers outputs fewer time-steps then they recieve, this option concatenates some of the input (left or right) to make the output of the layer have the same length as the input.
- `pad_input` also takes `[-1, 0, 1]` as possible values for left-, None and right-padding of the input.

`layer_funcs` is a `dict` holding pre-defined layer functions. Just take a look at this object to see some possible combinations of these options.


### Other Parameters

- `database` must be the `Database` object holding the data you want to use for training.
- `train_set` must be a `pandas.DataFrame` specifying some subset of the data.
- `gate_c`, `residuals_c` and `skips_c` are the number of channels to use for each of this convolutions
- `conv_kwargs` is a `dict` of parameters for the `torch.nn.Conv1d` objects.
- `lf` is the layer_function. It should be a `partial` packed in a tuple (otherwise it mysteriously disappear from the hyperparameters...)
- `layers` is a tuple of integers. Each integers `k` defines a block of `k` layers that will have `2**k` time-steps in its receptive-field.
- `learn_padding` lets you make the input padding learnable whenever you use `pad-input` in the layer-function.
- `lr` is the max learning rate. Values between 1e-3 and 1e-4 work in most cases.
- `batch_size` and `sequence_length` lets you specify `N` and `S` for batches of shape `(N x S x D)` aka (number of sequences, number of steps per sequence, time-step dimensionality)
- `root_dir`, `name` and `version` specify where to store the model files. The path to all the files will be `root_dir/name/version`. If you don't specify a `version` and `overwrite` is `False` a new folder is added in `root_dir/name`
- `era_duration` is the frequency at which you want to make training checkpoints. `era_duration=50` means you'll have one checkpoint every 50 epochs. 

In [ ]:
fnet = FreqNet(
    # data :
    database=db,
    train_set=db.metadata.iloc[[0, ]],
    input_dim=db.fft.dim,
    # model architecture
    model_dim=512,
    conv_kwargs=dict(groups=1),
    lf=layer_funcs["residuals_left"],
    layers=(int(np.log2(16)),),
    learn_padding=False,
    # training
    lr=5e-4,
    batch_size=64,
    sequence_length=64,
    max_epochs=100,
    # file-system
    overwrite=False,
    name="mmk_test_model",
    root_dir="mmk_test_model/",
    era_duration=50,
)

trainer = fnet.get_trainer()
trainer.fit(fnet)

## Generate from trained model

In [ ]:
root_dir="mmk_test_model/"
name="mmk_test_model"
version = str(0)
epoch = None

# load the checkpoint
fnet = FreqNet.load(FreqNet, root_dir + name + "/v" + version + "/", epoch)

input_length = 64
n_steps = 512
# pick a random input slice from the trainset
piece = fnet.train_set.sample(1)
input_start = np.random.randint(piece.start.min(), piece.stop.max(), 1)[0]

inpt = fnet.database.fft[input_start:input_start+input_length]

if torch.cuda.is_available():
    fnet.to("cuda")

generated = generate(fnet, inpt, n_steps, *fnet.generation_slices())

audio(generated.T)
show(generated.T)
